# 06 現実世界の応用
## Wandasを実際のデータ分析に活用する

このノートブックでは、**実際の応用事例**を通じてWandasの実践的な活用方法を紹介します。音声処理、振動分析、生体信号などの具体的なユースケースを通じて、Wandasがどのように価値を生み出すかを紹介します。

**学習目標:**
- 音声信号処理の実践
- 振動・機械診断
- センサーデータ分析
- 生体信号処理
- 産業応用事例

**前提条件:**
- 05_visualization_and_insights.ipynb を完了していること

## 🎯 なぜ現実世界の応用が重要か

### 理論から実践へ

**実際のデータ分析**では：

1. **ドメイン知識**が不可欠
2. **データの質**が結果を左右
3. **実用的制約**を考慮する必要
4. **ビジネス価値**を意識

### Wandasの強み

- **信号処理の専門性**: 音響・振動分野の深い理解
- **実用的API**: 研究から産業まで対応
- **品質保証**: 厳格なテストとドキュメント
- **拡張性**: カスタム処理の容易な追加

### 学習アプローチ

各事例で：
- **問題設定**: 何を解決するか
- **データ理解**: どのようなデータか
- **分析手法**: どのような手法を適用
- **結果解釈**: 何がわかったか
- **実践的示唆**: どのように活用するか

## 🎵 事例1: 音声信号処理 - 音質評価

### 問題設定

**音声品質評価システム**の開発：
- 録音デバイスの性能評価
- 音声圧縮の品質劣化検出
- 環境ノイズの影響評価
- 自動品質スコアリング

In [ ]:
# 必要なライブラリをインポート
import wandas as wd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from pathlib import Path

# インタラクティブプロット設定
%matplotlib widget
plt.rcParams['figure.figsize'] = (12, 6)

print(f"Wandas: {wd.__version__}")
print("✅ 準備完了")

In [ ]:
# サンプル音声データの準備
audio_samples = {}

# 1. クリーン音声（参照）
sr = 44100
duration = 3.0
time = np.linspace(0, duration, int(duration * sr))

# 複数の周波数成分を持つテスト信号
clean_audio = (
    0.5 * np.sin(2 * np.pi * 440 * time) +  # A4音
    0.3 * np.sin(2 * np.pi * 880 * time) +  # A5音
    0.2 * np.sin(2 * np.pi * 1320 * time)   # E6音
)

audio_samples['Clean'] = wd.from_numpy(
    data=clean_audio.reshape(1, -1),
    sampling_rate=sr,
    ch_labels=['Clean Audio']
)

# 2. ノイズ付加音声
noisy_audio = clean_audio + 0.1 * np.random.randn(len(clean_audio))
audio_samples['Noisy'] = wd.from_numpy(
    data=noisy_audio.reshape(1, -1),
    sampling_rate=sr,
    ch_labels=['Noisy Audio']
)

# 3. 低品質圧縮音声（シミュレーション）
compressed_audio = clean_audio.copy()
# 高周波成分を減衰させて圧縮をシミュレート
high_freq_mask = np.abs(np.fft.fftfreq(len(time), 1/sr)) > 2000
compressed_spectrum = np.fft.fft(compressed_audio)
compressed_spectrum[high_freq_mask] *= 0.3  # 高周波を大幅減衰
compressed_audio = np.real(np.fft.ifft(compressed_spectrum))

audio_samples['Compressed'] = wd.from_numpy(
    data=compressed_audio.reshape(1, -1),
    sampling_rate=sr,
    ch_labels=['Compressed Audio']
)

print(f"音声サンプル作成完了: {len(audio_samples)}種類")
for name, audio in audio_samples.items():
    print(f"  {name}: {audio.shape}, SNR: {10*np.log10(audio.rms[0]**2/np.var(audio.data.flatten())):.1f} dB")

In [ ]:
# 音質評価指標の計算
print("🎵 音質評価指標:")

quality_metrics = {}

for name, audio in audio_samples.items():
    # 基本指標
    rms = audio.rms[0]
    crest = audio.crest_factor[0]

    # 周波数領域指標
    spectrum = audio.fft()
    total_power = np.sum(np.abs(spectrum.data)**2)

    # 高周波成分のパワー（>2kHz）
    high_freq_mask = spectrum.freqs > 2000
    high_freq_power = np.sum(np.abs(spectrum.data[0, high_freq_mask])**2)
    hf_ratio = high_freq_power / total_power if total_power > 0 else 0

    # スペクトル平坦度（Shannon entropy風）
    power_spectrum = np.abs(spectrum.data[0])**2
    power_spectrum = power_spectrum / np.sum(power_spectrum)  # 正規化
    spectral_entropy = -np.sum(power_spectrum * np.log2(power_spectrum + 1e-10))

    quality_metrics[name] = {
        'RMS': rms,
        'Crest_Factor': crest,
        'High_Freq_Ratio': hf_ratio,
        'Spectral_Entropy': spectral_entropy,
        'Quality_Score': (rms * 0.3 + hf_ratio * 0.4 + (1 - spectral_entropy/10) * 0.3)  # 統合スコア
    }

# 結果表示
import pandas as pd
metrics_df = pd.DataFrame(quality_metrics).T
print(metrics_df.round(4))

In [ ]:
# 音質比較の可視化
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 時間波形
for name, audio in audio_samples.items():
    ax1.plot(audio.time[:5000], audio.data[0, :5000], label=name, alpha=0.8)
ax1.set_title('Time Domain Comparison')
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Amplitude')
ax1.legend()
ax1.grid(True, alpha=0.3)

# スペクトル
for name, audio in audio_samples.items():
    spectrum = audio.fft()
    ax2.semilogy(spectrum.freqs, np.abs(spectrum.data[0]), label=name, alpha=0.8)
ax2.set_title('Frequency Domain Comparison')
ax2.set_xlabel('Frequency [Hz]')
ax2.set_ylabel('Magnitude')
ax2.set_xlim(0, 5000)
ax2.legend()
ax2.grid(True, alpha=0.3)

# 品質スコア比較
quality_scores = [metrics['Quality_Score'] for metrics in quality_metrics.values()]
bars = ax3.bar(quality_metrics.keys(), quality_scores, color=['green', 'orange', 'red'], alpha=0.7)
ax3.set_title('Audio Quality Scores')
ax3.set_ylabel('Quality Score')
ax3.grid(True, alpha=0.3)

# 値をバー上に表示
for bar, score in zip(bars, quality_scores):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
            f'{score:.3f}', ha='center', va='bottom')

# スペクトルエントロピー
entropies = [metrics['Spectral_Entropy'] for metrics in quality_metrics.values()]
bars = ax4.bar(quality_metrics.keys(), entropies, color=['green', 'orange', 'red'], alpha=0.7)
ax4.set_title('Spectral Entropy (lower = more tonal)')
ax4.set_ylabel('Entropy')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**🎵 音質評価の洞察**

- **Clean**: 最も高い品質スコア、高周波成分が豊富
- **Noisy**: RMSは高いが高周波比が低下、スペクトルエントロピーが上昇
- **Compressed**: 高周波が大幅減衰、品質スコアが最低

**実践的示唆**:
- 音質監視システムにスペクトルエントロピーを活用
- 高周波比で圧縮劣化を検出
- 品質スコアで自動評価を実現

## 🔧 事例2: 振動分析 - 機械診断

### 問題設定

**回転機械の状態監視システム**：
- ベアリングの劣化検出
- インバランスの診断
- 共振周波数の特定
- 予防保全の最適化

In [ ]:
# 振動データのシミュレーション
vibration_sr = 10000  # 10kHz（振動測定に適した高サンプリングレート）
vibration_duration = 10.0
vibration_time = np.linspace(0, vibration_duration, int(vibration_duration * vibration_sr))

# 正常状態の振動
normal_vibration = (
    1.0 * np.sin(2 * np.pi * 30 * vibration_time) +     # 回転周波数（30Hz = 1800 RPM）
    0.3 * np.sin(2 * np.pi * 60 * vibration_time) +     # 2倍周波数
    0.1 * np.sin(2 * np.pi * 90 * vibration_time) +     # 3倍周波数
    0.05 * np.random.randn(len(vibration_time))        # 背景ノイズ
)

# ベアリング劣化状態（インパルス列）
bearing_fault = normal_vibration.copy()
fault_times = np.arange(0, vibration_duration, 1/30)  # 30Hzで繰り返す故障
for fault_time in fault_times:
    if fault_time < vibration_duration:
        idx = int(fault_time * vibration_sr)
        if idx < len(bearing_fault) - 100:
            # 減衰するインパルス
            impulse = 3.0 * np.exp(-np.arange(100) / 20)
            bearing_fault[idx:idx+100] += impulse

# インバランス状態（1倍周波数強調）
imbalance = (
    2.5 * np.sin(2 * np.pi * 30 * vibration_time) +     # 強調された1倍周波数
    0.3 * np.sin(2 * np.pi * 60 * vibration_time) +
    0.1 * np.sin(2 * np.pi * 90 * vibration_time) +
    0.05 * np.random.randn(len(vibration_time))
)

# ChannelFrame作成
vibration_conditions = wd.from_numpy(
    data=np.vstack([normal_vibration, bearing_fault, imbalance]),
    sampling_rate=vibration_sr,
    ch_labels=['Normal', 'Bearing_Fault', 'Imbalance']
)

print(f"振動データ作成完了: {vibration_conditions.shape}")
print(f"サンプリングレート: {vibration_conditions.sampling_rate} Hz")
print(f"条件: {vibration_conditions.labels}")

In [ ]:
# 機械診断指標の計算
print("🔧 機械診断指標:")

diagnostic_metrics = {}

for condition in vibration_conditions.labels:
    ch_data = vibration_conditions[condition]

    # RMS（全体振動レベル）
    rms = ch_data.rms[0]

    # クレストファクター（インパルス性の指標）
    crest = ch_data.crest_factor[0]

    # スペクトル分析
    spectrum = ch_data.welch(nperseg=4096, noverlap=2048)

    # 調和成分のパワー
    fundamental_power = np.abs(spectrum.data[0, np.argmin(np.abs(spectrum.freqs - 30))])
    second_harmonic_power = np.abs(spectrum.data[0, np.argmin(np.abs(spectrum.freqs - 60))])
    third_harmonic_power = np.abs(spectrum.data[0, np.argmin(np.abs(spectrum.freqs - 90))])

    # 調和指数
    total_harmonic_power = fundamental_power + second_harmonic_power + third_harmonic_power
    harmonic_index = second_harmonic_power / fundamental_power if fundamental_power > 0 else 0

    # 高周波エネルギー（ベアリング故障指標）
    high_freq_mask = (spectrum.freqs > 1000) & (spectrum.freqs < 3000)
    high_freq_energy = np.sum(np.abs(spectrum.data[0, high_freq_mask])**2)
    total_energy = np.sum(np.abs(spectrum.data[0])**2)
    hf_ratio = high_freq_energy / total_energy if total_energy > 0 else 0

    diagnostic_metrics[condition] = {
        'RMS': rms,
        'Crest_Factor': crest,
        'Harmonic_Index': harmonic_index,
        'High_Freq_Ratio': hf_ratio,
        'Diagnosis': (
            'Normal' if crest < 3 and harmonic_index < 0.5 and hf_ratio < 0.1 else
            'Bearing_Fault' if crest > 4 and hf_ratio > 0.15 else
            'Imbalance' if harmonic_index > 0.8 else
            'Unknown'
        )
    }

# 結果表示
diag_df = pd.DataFrame(diagnostic_metrics).T
print(diag_df.round(4))

In [ ]:
# 振動診断の可視化
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 時間波形（最初の1秒）
for condition in vibration_conditions.labels:
    ch_data = vibration_conditions[condition]
    time_subset = ch_data.time[:ch_data.sampling_rate]  # 1秒分
    data_subset = ch_data.data[0, :ch_data.sampling_rate]
    ax1.plot(time_subset, data_subset, label=condition, alpha=0.8)
ax1.set_title('Vibration Time Series (1s)')
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Acceleration')
ax1.legend()
ax1.grid(True, alpha=0.3)

# スペクトル比較
colors = ['green', 'red', 'orange']
for i, condition in enumerate(vibration_conditions.labels):
    ch_data = vibration_conditions[condition]
    spectrum = ch_data.welch(nperseg=4096, noverlap=2048)
    ax2.semilogy(spectrum.freqs, np.abs(spectrum.data[0]),
                label=condition, color=colors[i], alpha=0.8)
ax2.set_title('Vibration Spectra')
ax2.set_xlabel('Frequency [Hz]')
ax2.set_ylabel('Amplitude')
ax2.set_xlim(0, 200)
ax2.legend()
ax2.grid(True, alpha=0.3)

# 診断指標レーダー
categories = ['RMS', 'Crest_Factor', 'Harmonic_Index', 'High_Freq_Ratio']
angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]  # 閉じる

for i, condition in enumerate(vibration_conditions.labels):
    values = [diagnostic_metrics[condition][cat] for cat in categories]
    # 正規化（0-1スケール）
    if condition == 'Normal':
        norm_values = [v/3 for v in values]  # RMS基準
    else:
        norm_values = values
    norm_values += norm_values[:1]

    ax3.plot(angles, norm_values, 'o-', linewidth=2, label=condition, color=colors[i])
    ax3.fill(angles, norm_values, alpha=0.25, color=colors[i])

ax3.set_xticks(angles[:-1])
ax3.set_xticklabels(categories)
ax3.set_title('Diagnostic Metrics Radar')
ax3.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0))
ax3.grid(True, alpha=0.3)

# 診断結果
diagnoses = [diagnostic_metrics[cond]['Diagnosis'] for cond in vibration_conditions.labels]
bars = ax4.bar(vibration_conditions.labels, [1, 1, 1],
              color=['green', 'red', 'orange'], alpha=0.7)
ax4.set_title('Machine Diagnosis Results')
ax4.set_ylabel('Status')
ax4.set_ylim(0, 1.5)

# 診断結果をテキストで表示
for i, (bar, diagnosis) in enumerate(zip(bars, diagnoses)):
    ax4.text(bar.get_x() + bar.get_width()/2, 0.5,
            diagnosis, ha='center', va='center',
            fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

**🔧 振動診断の洞察**

- **Normal**: 低RMS、低クレストファクター、調和バランス良好
- **Bearing_Fault**: 高クレストファクター、高周波エネルギー増加
- **Imbalance**: 1倍周波数強調、調和指数上昇

**実践的示唆**:
- クレストファクターでインパルス故障を検出
- 高周波比でベアリング劣化を監視
- 調和指数でインバランスを診断
- レーダーチャートで総合的な機械状態を把握

## 🫀 事例3: 生体信号処理 - 心拍変動分析

### 問題設定

**心拍変動（HRV）分析システム**：
- ストレスレベルの評価
- 自律神経機能の評価
- 睡眠品質の評価
- 健康状態のモニタリング

In [ ]:
# 心拍データのシミュレーション
hrv_sr = 250  # 250Hz（ECGに適したサンプリングレート）
hrv_duration = 60.0  # 1分間
hrv_time = np.linspace(0, hrv_duration, int(hrv_duration * hrv_sr))

# 正常心拍（安静時）
rr_intervals_normal = np.random.normal(0.8, 0.05, 75)  # 75拍分（1分間）
rr_times_normal = np.cumsum(rr_intervals_normal)
rr_times_normal = rr_times_normal[rr_times_normal < hrv_duration]

# ストレス時心拍（変動減少）
rr_intervals_stress = np.random.normal(0.7, 0.02, 85)  # 心拍数上昇
rr_times_stress = np.cumsum(rr_intervals_stress)
rr_times_stress = rr_times_stress[rr_times_stress < hrv_duration]

# 運動時心拍（変動増加）
rr_intervals_exercise = np.random.normal(0.5, 0.08, 120)  # 心拍数大幅上昇
rr_times_exercise = np.cumsum(rr_intervals_exercise)
rr_times_exercise = rr_times_exercise[rr_times_exercise < hrv_duration]

# R波位置からECG波形を生成
def generate_ecg_signal(rr_times, duration, sampling_rate):
    """R波位置からECG波形を生成"""
    time = np.linspace(0, duration, int(duration * sampling_rate))
    ecg = np.zeros_like(time)

    # 各R波に対してECG波形を追加
    for rr_time in rr_times:
        if rr_time < duration:
            idx = int(rr_time * sampling_rate)
            if idx < len(time) - 100:
                # 簡易ECG波形（P波, QRS波, T波）
                t_local = np.arange(100) / sampling_rate
                # P波
                p_wave = 0.1 * np.exp(-((t_local - 0.1)**2) / (2 * 0.01**2))
                # QRS波
                qrs_wave = -0.2 * np.exp(-((t_local - 0.15)**2) / (2 * 0.005**2)) + \
                          1.0 * np.exp(-((t_local - 0.16)**2) / (2 * 0.003**2)) + \
                          -0.3 * np.exp(-((t_local - 0.17)**2) / (2 * 0.005**2))
                # T波
                t_wave = 0.2 * np.exp(-((t_local - 0.35)**2) / (2 * 0.02**2))

                ecg_wave = p_wave + qrs_wave + t_wave
                ecg[idx:idx+100] += ecg_wave

    # ベースラインノイズと筋電図ノイズを追加
    ecg += 0.01 * np.random.randn(len(ecg))  # ベースライン
    # バンドパスフィルタで筋電図をシミュレート
    emg_noise = np.random.randn(len(ecg))
    from scipy import signal
    b, a = signal.butter(4, [20, 150], btype='band', fs=sampling_rate)
    emg_filtered = signal.filtfilt(b, a, emg_noise)
    ecg += 0.02 * emg_filtered

    return ecg

# ECG信号生成
ecg_normal = generate_ecg_signal(rr_times_normal, hrv_duration, hrv_sr)
ecg_stress = generate_ecg_signal(rr_times_stress, hrv_duration, hrv_sr)
ecg_exercise = generate_ecg_signal(rr_times_exercise, hrv_duration, hrv_sr)

# ChannelFrame作成
ecg_conditions = wd.from_numpy(
    data=np.vstack([ecg_normal, ecg_stress, ecg_exercise]),
    sampling_rate=hrv_sr,
    ch_labels=['Rest', 'Stress', 'Exercise']
)

print(f"ECGデータ作成完了: {ecg_conditions.shape}")
print(f"条件: {ecg_conditions.labels}")
print(f"心拍数: Rest={len(rr_times_normal):.0f} bpm, Stress={len(rr_times_stress):.0f} bpm, Exercise={len(rr_times_exercise):.0f} bpm")

In [ ]:
# HRV分析指標の計算
print("🫀 HRV分析指標:")

hrv_metrics = {}

# R波検出とRR間隔計算（簡易版）
def detect_r_peaks(ecg_signal, sampling_rate):
    """簡易R波検出（実際にはより洗練されたアルゴリズムが必要）"""
    # バンドパスフィルタ（5-15Hz）
    from scipy import signal
    b, a = signal.butter(4, [5, 15], btype='band', fs=sampling_rate)
    filtered = signal.filtfilt(b, a, ecg_signal)

    # 微分と二乗
    diff = np.diff(filtered)
    squared = diff ** 2

    # 移動平均
    window_size = int(0.15 * sampling_rate)  # 150ms
    averaged = np.convolve(squared, np.ones(window_size)/window_size, mode='same')

    # 閾値処理
    threshold = np.mean(averaged) + 2 * np.std(averaged)
    peaks = []
    min_distance = int(0.4 * sampling_rate)  # 400ms
    last_peak = -min_distance

    for i in range(len(averaged)):
        if averaged[i] > threshold and (i - last_peak) > min_distance:
            peaks.append(i)
            last_peak = i

    return np.array(peaks) / sampling_rate  # 時間を返す

for condition in ecg_conditions.labels:
    ch_data = ecg_conditions[condition]

    # R波検出
    r_peaks = detect_r_peaks(ch_data.data[0], ch_data.sampling_rate)

    if len(r_peaks) > 2:
        # RR間隔計算
        rr_intervals = np.diff(r_peaks)

        # 時間領域指標
        mean_rr = np.mean(rr_intervals)
        sdnn = np.std(rr_intervals)  # SDNN
        rmssd = np.sqrt(np.mean(np.diff(rr_intervals)**2))  # RMSSD

        # 心拍数変動係数
        cv_rr = sdnn / mean_rr if mean_rr > 0 else 0

        # 周波数領域指標（簡易版）
        # RR間隔をリサンプリングしてスペクトル分析
        rr_resampled = np.interp(
            np.linspace(0, len(rr_intervals)-1, len(rr_intervals)*10),
            np.arange(len(rr_intervals)),
            rr_intervals
        )

        # 線形トレンド除去
        rr_detrended = signal.detrend(rr_resampled)

        # ハミング窓適用
        windowed = rr_detrended * np.hamming(len(rr_detrended))

        # FFT
        rr_spectrum = np.abs(np.fft.fft(windowed))
        freqs = np.fft.fftfreq(len(windowed), 1/(10 * 4))  # 4Hzサンプリング相当

        # VLF, LF, HFパワー
        vlf_mask = (freqs >= 0.0033) & (freqs < 0.04)
        lf_mask = (freqs >= 0.04) & (freqs < 0.15)
        hf_mask = (freqs >= 0.15) & (freqs < 0.4)

        vlf_power = np.sum(rr_spectrum[vlf_mask]**2) if np.any(vlf_mask) else 0
        lf_power = np.sum(rr_spectrum[lf_mask]**2) if np.any(lf_mask) else 0
        hf_power = np.sum(rr_spectrum[hf_mask]**2) if np.any(hf_mask) else 0

        total_power = vlf_power + lf_power + hf_power
        lf_hf_ratio = lf_power / hf_power if hf_power > 0 else 0

        hrv_metrics[condition] = {
            'Heart_Rate': 60 / mean_rr,
            'SDNN': sdnn * 1000,  # ms
            'RMSSD': rmssd * 1000,  # ms
            'CV_RR': cv_rr,
            'LF_HF_Ratio': lf_hf_ratio,
            'Total_Power': total_power,
            'Stress_Level': (
                'Low' if cv_rr > 0.05 and lf_hf_ratio < 2 else
                'Medium' if cv_rr > 0.03 else
                'High'
            )
        }
    else:
        hrv_metrics[condition] = {
            'Heart_Rate': 0,
            'SDNN': 0,
            'RMSSD': 0,
            'CV_RR': 0,
            'LF_HF_Ratio': 0,
            'Total_Power': 0,
            'Stress_Level': 'Unknown'
        }

# 結果表示
hrv_df = pd.DataFrame(hrv_metrics).T
print(hrv_df.round(4))

In [ ]:
# HRV分析の可視化
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# ECG波形（最初の5秒）
for condition in ecg_conditions.labels:
    ch_data = ecg_conditions[condition]
    time_subset = ch_data.time[:int(5 * ch_data.sampling_rate)]  # 5秒分
    data_subset = ch_data.data[0, :int(5 * ch_data.sampling_rate)]
    ax1.plot(time_subset, data_subset, label=condition, alpha=0.8)
ax1.set_title('ECG Signals (5s)')
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Amplitude [mV]')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 心拍数比較
heart_rates = [hrv_metrics[cond]['Heart_Rate'] for cond in ecg_conditions.labels]
bars = ax2.bar(ecg_conditions.labels, heart_rates,
              color=['blue', 'red', 'green'], alpha=0.7)
ax2.set_title('Heart Rate Comparison')
ax2.set_ylabel('BPM')
ax2.grid(True, alpha=0.3)

for bar, hr in zip(bars, heart_rates):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
            f'{hr:.1f}', ha='center', va='bottom')

# HRV指標レーダー
categories = ['SDNN', 'RMSSD', 'CV_RR', 'LF_HF_Ratio']
angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]

colors = ['blue', 'red', 'green']
for i, condition in enumerate(ecg_conditions.labels):
    values = [hrv_metrics[condition][cat] for cat in categories]
    # 正規化
    norm_values = [
        v/100 if cat == 'SDNN' else
        v/50 if cat == 'RMSSD' else
        v/0.1 if cat == 'CV_RR' else
        min(v/5, 1)  # LF_HF_Ratio
        for v, cat in zip(values, categories)
    ]
    norm_values += norm_values[:1]

    ax3.plot(angles, norm_values, 'o-', linewidth=2, label=condition, color=colors[i])
    ax3.fill(angles, norm_values, alpha=0.25, color=colors[i])

ax3.set_xticks(angles[:-1])
ax3.set_xticklabels(categories)
ax3.set_title('HRV Metrics Radar')
ax3.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax3.grid(True, alpha=0.3)

# ストレスレベル診断
stress_levels = [hrv_metrics[cond]['Stress_Level'] for cond in ecg_conditions.labels]
colors_stress = ['green' if level == 'Low' else 'orange' if level == 'Medium' else 'red'
                 for level in stress_levels]
bars = ax4.bar(ecg_conditions.labels, [1, 1, 1], color=colors_stress, alpha=0.7)
ax4.set_title('Stress Level Assessment')
ax4.set_ylabel('Status')
ax4.set_ylim(0, 1.5)

for i, (bar, level) in enumerate(zip(bars, stress_levels)):
    ax4.text(bar.get_x() + bar.get_width()/2, 0.5,
            level, ha='center', va='center',
            fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

**🫀 HRV分析の洞察**

- **Rest**: 正常なHRV、低ストレス状態
- **Stress**: HRV減少、交感神経優位
- **Exercise**: 心拍数上昇、変動増加

**実践的示唆**:
- RMSSDで副交感神経活性を評価
- LF/HF比で自律神経バランスを把握
- SDNNで全体的なHRVを評価
- ストレスモニタリングへの活用

## 🚀 統合された産業応用ワークフロー

### クロスドメイン分析

異なる分野のデータを統合して分析する完全ワークフローを体験しましょう。

In [ ]:
# 統合分析ワークフロー
print("🚀 クロスドメイン統合分析ワークフロー:")

# 1. マルチモーダルデータ統合
print("1. 📊 マルチモーダルデータ統合")

# 各ドメインの代表的なデータを統合
integrated_data = wd.FrameDataset([
    audio_samples['Clean'],
    vibration_conditions['Normal'],
    ecg_conditions['Rest']
], labels=['Audio_Quality', 'Vibration_Normal', 'ECG_Rest'])

print(f"統合データセット: {len(integrated_data)} チャンネル")
for i, frame in enumerate(integrated_data.frames):
    print(f"  {integrated_data.labels[i]}: {frame.shape}, {frame.sampling_rate} Hz")

# 2. 共通指標の計算
print("\n2. 📈 共通指標の計算")

common_metrics = {}
for name, frame in zip(integrated_data.labels, integrated_data.frames):
    spectrum = frame.fft()

    # スペクトル重心（周波数分布の中心）
    freq_centroid = np.sum(spectrum.freqs * np.abs(spectrum.data[0])**2) / np.sum(np.abs(spectrum.data[0])**2)

    # スペクトルロールオフ（エネルギーの95%を含む周波数）
    cumsum = np.cumsum(np.abs(spectrum.data[0])**2)
    rolloff_idx = np.where(cumsum >= 0.95 * cumsum[-1])[0][0]
    rolloff_freq = spectrum.freqs[rolloff_idx]

    # スペクトルフラットネス（トーナル vs ノイズ性）
    spectral_flatness = np.exp(np.mean(np.log(np.abs(spectrum.data[0]) + 1e-10))) / np.mean(np.abs(spectrum.data[0]))

    common_metrics[name] = {
        'RMS': frame.rms[0],
        'Freq_Centroid': freq_centroid,
        'Spectral_Rolloff': rolloff_freq,
        'Spectral_Flatness': spectral_flatness,
        'Domain': (
            'Audio' if 'Audio' in name else
            'Vibration' if 'Vibration' in name else
            'ECG' if 'ECG' in name else 'Unknown'
        )
    }

metrics_df = pd.DataFrame(common_metrics).T
print(metrics_df.round(4))

# 3. ドメイン横断的な洞察
print("\n3. 💡 ドメイン横断的な洞察")

insights = [
    "スペクトル重心: Audio > ECG > Vibration（高周波 vs 低周波の違い）",
    "スペクトルフラットネス: Audioが最もトーナル、Vibrationが最もノイズ的",
    "RMSレベル: 各ドメインで適切なスケーリングが必要",
    "共通の信号処理手法が異なるドメインで有効",
    "ドメイン固有の特徴量と共通指標の組み合わせが効果的"
]

for i, insight in enumerate(insights, 1):
    print(f"   {i}. {insight}")

# 4. 統合可視化
print("\n4. 📊 統合可視化")
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 時間波形比較（正規化）
for name, frame in zip(integrated_data.labels, integrated_data.frames):
    normalized_data = frame.data[0] / frame.rms[0]  # RMSで正規化
    time_subset = frame.time[:min(len(frame.time), 1000)]  # 最初の1000サンプル
    data_subset = normalized_data[:1000]
    ax1.plot(time_subset, data_subset, label=name, alpha=0.8)
ax1.set_title('Normalized Time Series Comparison')
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Normalized Amplitude')
ax1.legend()
ax1.grid(True, alpha=0.3)

# スペクトル比較
for name, frame in zip(integrated_data.labels, integrated_data.frames):
    spectrum = frame.fft()
    ax2.semilogy(spectrum.freqs, np.abs(spectrum.data[0]), label=name, alpha=0.8)
ax2.set_title('Spectral Comparison')
ax2.set_xlabel('Frequency [Hz]')
ax2.set_ylabel('Magnitude')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 共通指標レーダー
categories = ['RMS', 'Freq_Centroid', 'Spectral_Rolloff', 'Spectral_Flatness']
angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]

colors = ['purple', 'orange', 'cyan']
for i, name in enumerate(integrated_data.labels):
    values = [common_metrics[name][cat] for cat in categories]
    # 正規化
    norm_values = [
        v/2 if cat == 'RMS' else
        v/1000 if cat == 'Freq_Centroid' else
        v/2000 if cat == 'Spectral_Rolloff' else
        v/0.5  # Spectral_Flatness
        for v, cat in zip(values, categories)
    ]
    norm_values += norm_values[:1]

    ax3.plot(angles, norm_values, 'o-', linewidth=2, label=name, color=colors[i])
    ax3.fill(angles, norm_values, alpha=0.25, color=colors[i])

ax3.set_xticks(angles[:-1])
ax3.set_xticklabels(categories)
ax3.set_title('Cross-Domain Metrics Comparison')
ax3.legend(loc='upper right', bbox_to_anchor=(1.4, 1.0))
ax3.grid(True, alpha=0.3)

# ドメイン分類
domains = [common_metrics[name]['Domain'] for name in integrated_data.labels]
domain_colors = ['blue' if d == 'Audio' else 'green' if d == 'Vibration' else 'red' for d in domains]
bars = ax4.bar(integrated_data.labels, [1, 1, 1], color=domain_colors, alpha=0.7)
ax4.set_title('Domain Classification')
ax4.set_ylabel('Domain')
ax4.set_ylim(0, 1.5)

for i, (bar, domain) in enumerate(zip(bars, domains)):
    ax4.text(bar.get_x() + bar.get_width()/2, 0.5,
            domain, ha='center', va='center',
            fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✅ クロスドメイン統合分析ワークフロー完了！")

## 🛠️ トラブルシューティング

### 現実応用でのよくある問題

#### 1. データ品質の問題
- **センサーノイズ**: 適切なフィルタリングとキャリブレーション
- **欠損データ**: 補間手法（線形、キュービック、スプライン）
- **同期ずれ**: クロック同期とタイムスタンプ管理

#### 2. ドメイン知識の不足
- **専門家との協働**: 各分野の専門家と連携
- **文献調査**: ドメイン固有のベストプラクティスを学ぶ
- **検証実験**: 理論と実測のギャップを埋める

#### 3. スケーラビリティ
- **大規模データ**: Daskや分散処理の活用
- **リアルタイム処理**: ストリーミング処理とオンラインアルゴリズム
- **メモリ管理**: データの段階的処理とキャッシュ戦略

#### 4. 実用化の課題
- **堅牢性**: エッジケースと異常値への対応
- **保守性**: コードのドキュメントとテスト
- **ユーザビリティ**: 直感的なインターフェースとエラーメッセージ

## 📚 次のステップ

現実世界の応用をマスターしました！

**次のノートブック**: [07_conclusion_and_next_steps.ipynb](07_conclusion_and_next_steps.ipynb)

ここでは、学習した内容を振り返り、今後の学習パスについて考えます。

### 🎯 これまでに学んだこと
- ✅ 音声信号処理（品質評価、スペクトル分析）
- ✅ 振動分析（機械診断、故障検知）
- ✅ 生体信号処理（HRV分析、自律神経評価）
- ✅ クロスドメイン統合分析
- ✅ 実践的な信号処理ワークフロー

### 🚀 次の学習目標
- 機械学習との統合
- ストリーミングデータ処理
- 分散処理と大規模データ
- カスタムアルゴリズム開発
- 産業システムへの展開

---

**現実世界の信号処理を身につけました。Wandasで何でも解決しましょう！** 🎯